In [164]:
import cv2
import math
import numpy as np
import pandas as pd
from PIL import Image
from scipy.optimize import minimize, Bounds
from scipy.signal import deconvolve, fftconvolve
from scipy import fftpack
import plotly.express as px

In [165]:
from util.fft import *
from util.blurs import *
from util.images import *
from util.borders import *

In [166]:
def psf_linear(dim: tuple, r: float, theta: float, i: int = None, j: int = None) -> np.ndarray:
    '''
    Returns a point source function blurred with a linear blur. 

    Parameters:
        dim (tuple): dimensions of the image to produce, ex. (3, 3)
        r (float): length of blur in pixels
        theta (float): angle of blur in radians
        i (int, optional): offset for center of psf in dim[0]
        j (int, optional): offset for center of psf in dim[1]

    Returns:
        Normalized matrix for linear blur.
    '''

    # verify that dimensions are greater than 0
    if dim[0] < 1 or dim[1] < 1:
        raise ValueError("Dimensions cannot be less than 1.") 

    # if i and j were not assigned, manually assign them to center of image
    if i is None:
        i = math.ceil(dim[0] / 2)
    if j is None:
        j = math.ceil(dim[0] / 2)

    # verify that offsets are between 0 and dim
    if i < 0:
        i = 0
    if i >= dim[0]:
        i = dim[0] - 1
    if j < 0:
        j = 0
    if j >= dim[1]:
        j = dim[1] - 1



    # create matrix of zeros
    p = np.zeros(dim)

    for radius in range(0, math.floor(r)+1):
        px = math.floor(radius * math.cos(theta) + i) % dim[0]
        py = math.floor(-radius * math.sin(theta) + j) % dim[1]
        p[py, px] = 1

    # normalize p values to [0, 1]
    # p = p / np.linalg.norm(p)

    return p

In [167]:
from math import inf

def tik_deblur_fft(B, P, center, alpha = 0):
    # compute eigenvalues
    S = fft2(circshift(P, (0 - center[0], 0 - center[1])))
    # fix for eigenvalues equal to zero
    S = np.where(S != 0, S, float('inf'))
    s = S.flatten() # should be col-wise
    # calculate regularizaton parameter
    bhat = fft2(B)
    bhat = bhat.flatten()
    # TODO: graph results of varying alphas, compute error between true image and deblurred image
    # alpha = gcv_tik(s, bhat)
    # compute Tikhonov regularized solution
    s = np.where(s != inf, s, np.finfo(np.float64).eps)
    D = s.conj() * s + abs(alpha)**2
    bhat = s.conj() * bhat
    xhat = np.divide(bhat, D)
    xhat = np.reshape(xhat, B.shape)
    x = np.real(ifft2(xhat))
    return x

In [168]:
psf_size   = (32, 32)             # dimensions of PSF
psf_fn     = psf_gaussian         # specifies function to generate PSF
s          = 0.1                  # spread of PSF; s = s1 = s2
length     = 3
angle      = 0
noise      = 0.00                 # amount of noise to add to blurred image
border_fn  = periodic             # specifies boundary conditions
blur_fn    = blur_fft             # specifies how to produce a blur
deblur_fn  = tik_deblur_fft       # specifies how to deconvolute a blur
img_name   = 'shapes'             # image to use
sample_img = 'samples/{}.png'.format(img_name)
psf_center = tuple(int(i/2) for i in psf_size)

In [169]:
# load true image from path, values are normalized to [0, 1]
true_image = load_img(sample_img)

# apply boundary conditions
boundary_image = border_fn(true_image, tuple(i * 2 for i in psf_size))
save_img_raw(boundary_image, 'results/1_{}_boundary.png'.format(img_name))

# create the PSF
psf = blur(psf_fn, psf_size, s, s, psf_center[0], psf_center[1])
# psf = blur(psf_fn, psf_size, length, angle, psf_center[0], psf_center[1])
save_img_raw(psf, 'results/2_{}_psf.png'.format(img_name))

# pad PSF to the size of the image
psf_padded = pad_psf(psf, boundary_image.shape)

# blur the true image
blurred_true_image = blur_fn(boundary_image, psf_padded, psf_center)
save_img_raw(blurred_true_image, 'results/3_{}_true_blurred.png'.format(img_name))

# add noise to the blurred true image
blurred_true_image = add_noise(blurred_true_image, noise)
save_img_raw(blurred_true_image, 'results/4_{}_noisy_blurred.png'.format(img_name))

# deblur using tikhonov
deblurred_image = tik_deblur_fft(blurred_true_image, psf_padded, psf_center)
# deblurred_image = deblur_deconvolvefft(blurred_true_image, psf_padded)
save_img_raw(deblurred_image, 'results/5_{}_deblurred.png'.format(img_name))

#
# section 5.3 - SVD analysis    
#

# get the singular values of the PSF
svd_u, singular_values, svd_v = np.linalg.svd(psf)
df = pd.DataFrame(dict(
    i = [i for i in range(len(singular_values))],
    s = singular_values
))
fig = px.line(df, x = 'i', y = 's', title = 'Singular Value Decay of Deblurred Image')
fig.update_layout(
    xaxis_title = 'i',
    yaxis_title = 'Singular Value at i'
)
# fig.show()

# fig.add_scatter(x = [i for i in range(len(singular_values))], 
#                 y = [svd_u[i].transpose() * deblurred_image for i in range(len(singular_values))])
# for i in range(len(singular_values)):
#     u_i = svd_u[i].transpose()
    